In [ ]:
pip install python-dotenv


In [ ]:
import os
from tqdm.autonotebook import tqdm
import io
import codecs
import numpy as np
import multiprocessing

In [ ]:
from dotenv import load_dotenv

load_dotenv()


In [11]:
import pinecone
api_key=os.environ.get('Pinecone_API')


pinecone.init(api_key, environment="asia-northeast1-gcp")
pinecone.create_index(
 'open-ai',
    dimension=1536,
 shards=1)





In [12]:
index= pinecone.Index('open-ai')

In [14]:
import openai
openai.api_key= os.environ.get('openai_api')
model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        
    ], engine=model
)


In [ ]:
import gdown
url = 'https://drive.google.com/drive/folders/18nLtKaO5hYKfCxSgN8T3qEe9DxyAcnZA?usp=sharing'
output_path = 'C:/Users/Akshay/Desktop/filetesting'

gdown.download_folder(url=url, output=output_path, quiet=True)


In [ ]:
folder_path = "C:/Users/Akshay/Desktop/filetesting"  # Replace with the actual path to the folder

# Get a list of all files in the folder
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Print the list of files
print(file_list)

In [ ]:
# Define function to vectorize file using OpenAI
batch_size= 64
model = "text-embedding-ada-002"
def vectorize_file(file_path):
    with open(file_path, "r", encoding="latin1") as f:
        lines = f.readlines()
        from tqdm.auto import tqdm
        for i in tqdm(range(0, len(lines), batch_size)):
             lines_batch = lines[i: i+batch_size] 
             print(lines_batch)
             res = openai.Embedding.create(input=lines_batch, engine=model) 
             vector = res["data"][0]["embedding"]
             namespace = os.path.basename(file_path)  # use file name as namespace
             embeds = [record['embedding'] for record in res['data']]
             meta = [{'text': line} for line in lines_batch]
             to_upsert = zip(namespace, embeds, meta)
             index.upsert(vectors=list(to_upsert))
             print(namespace)

             print(embeds)
        
folder_path = "C:/Users/Akshay/Desktop/filetesting"  # Replace with the actual path to the folder

# Get a list of all files in the folder
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Print the list of files
print(file_list)
batch_size=64
for file_name in file_list:
    path_parts = ["C:/", "Users", "Akshay", "Desktop", "filetesting"]  # Replace with the path parts

# Use os.path.join to construct the full path
    file_path = os.path.join(*path_parts, file_name)

# Replace any backward slashes with forward slashes
    file_path = file_path.replace("\\", "/")

# Print the full path
    print(file_path)
    vector = vectorize_file(file_path)







In [ ]:
print("input_query")
query = "RPM"

xq = openai.Embedding.create(input=query, engine=model)['data'][0]['embedding']

res = index.query([xq], top_k=10, include_metadata=True)
for match in res['matches']:
    print(f"Namespace: {match['id']}")
    print(f"Score: {match['score']:.2f}")
    print(f"Text: {match['metadata']['text']}\n")

In [ ]:
index_stats_response = index.describe_index_stats()
print(index_stats_response)


In [15]:
pinecone.delete_index("open-ai")



In [35]:
folder = "C:/Users/Akshay/Desktop/filetesting"

# loop over each file in the folder
for filename in os.listdir(folder):
    # construct the full path of the file
    file_path = os.path.join(folder, filename)
    # check if the file is a regular file (i.e., not a directory)
    if os.path.isfile(file_path):
        # delete the file
        os.remove(file_path)